In [2]:
import pandas as pd
from hazm import stopwords_list, word_tokenize

In [ ]:
df = df[~df["NAME"].str.isdigit()]

In [ ]:
def drop_terminal_names_starting_with_english_words(dataframe, column_name):
    english_word_pattern = r'^[A-Za-z]+'

    mask = dataframe[column_name].str.contains(english_word_pattern, case=False, regex=True)

    dataframe = dataframe[~mask]

    return dataframe
filtered_df = drop_terminal_names_starting_with_english_words(df, 'NAME')

In [ ]:
def remove_persian_numbers(terminal_name):
    persian_numbers = {'۰': '', '۱': '', '۲': '', '۳': '', '۴': '', '۵': '', '۶': '', '۷': '', '۸': '', '۹': ''}
    pattern = re.compile('|'.join(persian_numbers.keys()))
    result = pattern.sub(lambda x: persian_numbers[x.group()], terminal_name)
    return result
filtered_df['NAME'] = filtered_df['NAME'].apply(remove_persian_numbers)

In [ ]:
persian_stopwords = stopwords_list()

In [ ]:
processed_terminal_names = []
for terminal_name in filtered_df['NAME']:
    tokens = word_tokenize(terminal_name)
    
    filtered_tokens = [word for word in tokens if word not in persian_stopwords]
    
    processed_terminal_name = ' '.join(filtered_tokens)
    
    processed_terminal_names.append(processed_terminal_name)
filtered_df['processed_name'] =processed_terminal_names

In [ ]:
def remove_persian_numbers_from_column(df, column_name):
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame")

    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame")

    persian_number_pattern = '[۰-۹]'

    df[column_name] = df[column_name].apply(lambda text: re.sub(persian_number_pattern, '', str(text)))

    return df
filtered_df = remove_persian_numbers_from_column(filtered_df, 'TERMINAL_NAME')

In [ ]:
def remove_emojis_and_symbols(text):
    emoji_symbol_pattern = re.compile("["
                                      u"\U0001F600-\U0001F64F"  # Emoticons
                                      u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                                      u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                                      u"\U0001F700-\U0001F77F"  # Alphanumeric & geometric shapes
                                      u"\U0001F780-\U0001F7FF"  # Geometric shapes extended
                                      u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                      u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                      u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                      u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                      u"\U00002702-\U000027B0"  # Dingbats
                                      u"\U000024C2-\U0001F251" 
                                      r"\W"  # Non-word characters (symbols)
                                      "]+", flags=re.UNICODE)

    # Remove emojis and symbols from the text
    clean_text = re.sub(emoji_symbol_pattern, ' ', text)

    return clean_text
filtered_df['cleaned_name'] = filtered_df['processed_name'].apply(remove_emojis_and_symbols)